# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

O problema se baseia em implementar um modelo matemático, utilizando programação dinâmica, que maximize a recompensa esperada e a probabilidade de ganhar uma determinada quantia de dinheiro no jogo de TV "Quem quer ser um milionário?"

Para esta implementação foi utilizado o artigo a seguir como referência: "Perea, F., & Puerto, J. (2007). Dynamic programming analysis of the TV game “Who wants to be a millionaire?”. European Journal of Operational Research, 183(2), 805-811.".

O competidor deve responder corretamente 15 questões de múltipla escolha em seguida. Cada questão possui 4 opções. O competidor pode parar, responder ou pedir ajuda. Há 3 tipos de ajuda:
* 50:50 que consiste em retirar 2 de 3 opções incorretas;
* Ligar para um amigo, onde terão 30 segundos para discussão; ou
* Plateia, onde há uma votação e o resultado porcentual é mostrado ao competidor.

## Modelagem Matemática

### Parâmetros

* $N$: estágios do jogo. Há 16 estágios, o 16º estágio indica que o competidor acertou a 15ª questão;
* $S$: conjunto de vetores de estado;
* $A(s)$: conjunto de ações viáveis no estado $s$;
* $r_{k}$: recompensa do competidor se ele decidir parar após responder a questão $k$ corretamente;
* $r_{k}^*$: recompensa do competidor se ele errar a questão $k+1$;
* $c_{k}^i$: fator fixo de melhoria ao usar a ajuda $i$ na questão $k$.

### Variáveis

* $k$: índice da questão atual;
* $l_{i}$: variável binária, onde é $1$ se a ajuda $i$ pode ser usada, $0$ caso contrário;
* $l_{i}^´$: variável binária, onde é $l_{i}-1$ se a ajuda $i$ é usada na questão $k$, $l_{i}$ caso contrário;
* $p_{k}^*$: probabilidade da resposta estar correta sem nenhuma ajuda na questão $k$;
* $p_{k}^i$: probabilidade da resposta estar correta utilizando a ajuda $i$ na questão $k$;
* $p_{s}^a$: probabilidade de acertar a resposta de uma questão no estado $s \in S$ com a escolha da ação $a \in A(s)$.

### Considerações

Se $k \le 15$, o competidor tem as seguintes ações:
1. Responder a questão sem ajuda;
2. Responder a questão utilizando uma ou mais ajuda(s);
3. Parar e sair do jogo (ganhando o valor do estágio anterior).

Alguns pontos que valem a pena serem ressaltados/relembrados:
* Se o competidor decidir responder, a recompensa é uma variável aleatória e depende da probabilidade de se responder certo;
* Se o competidor errar, ele ganhará a recompensa do último "ponto de garantia", que são as questões 5 e 10;
* Se o competidor parar ou errar, o jogo acaba;
* Se o competidor acertar, ele avança para o próximo estágio.

### Explicando melhor alguns tópicos da modelagem

No artigo, foi retratado a utilização de uma análise de regressão linear restrita que resultou em como representar as probabilidades $p_{k}^*, p_{k}^1,p_{k}^2 \ \text{e} \ p_{k}^3$:
* $p_{k}^* = 0.996 - 0.051(k - 1)$
* $p_{k}^1 = 1.000 - 0.037(k - 1)$
* $p_{k}^2 = 1.000 - 0.029(k - 1)$
* $p_{k}^3 = 1.000 - 0.041(k - 1)$

Ademais, o artigo discute outra forma de calcular $p_{k}^i$ que consiste em assumir que existe uma relação multiplicativa entre a probabilidade de falhar em um dado estado usando a ajuda $i$ e a probabilidade de falha sem nenhuma ajuda. Matematicamente:
$q_{k}^i = q_{k}^* ⋅ c_{k}^i ⇔ p_{k}^i = 1 - (1 - p_{k}^*) ⋅ c_{k}^i$.

Na implementação utilizou-se a fórmula $p_{k}^i$ pela relação multiplicativa, ao invés da análise de regressão linear.

Caso o competidor utilize simultaneamente duas ou mais ajudas na mesma questão, multiplica-se sucessivamente os fatores $c_{k}^i$. Matematicamente: $p_{k}^i = 1 - (1 - p_{k}^*) ⋅ c_{k}^1 ⋅ c_{k}^2 ⋅ \ \text{...} \ ⋅ c_{k}^n$.

### Função Objetivo

$f(s)$ consiste em maximizar a recompensa máxima esperada a partir de um estado.

Um estado é representado como $s = (k, l_{1}, l_{2}, l_{3})$, elemento do conjunto $S$.

Podemos avaliar a função objetivo da seguinte maneira:
* A recompensa máxima esperada de $s$ é a máxima entre as recompensas esperadas de todos os estados possíveis que podem ser alcançados a partir de $s$;
* Neste ponto, podemos sair do jogo e garantir $r_{k}$ ou ir para a próxima questão $(k+1)$;
* No último caso, se escolhermos a ação $a \in A(s)$ correta com probabilidade $p_{s}^a$ ou a errada com probabilidade $(1 - p_{s}^a)$, temos a recompensa $r_{k}^*$ do ponto de garantia se a resposta for incorreta, caso contrário iremos transitar para o próximo estágio;
* $t(s, a)=(k+1, l_{1}^´, l_{2}^´, l_{3}^´) \in S$ representa o avanço ao próximo estágio ao escolher a ação $a$ no estado $s$, ou seja, a recompensa se torna $f(t(s, a))$.

Sumarizando matematicamente, temos:

$f(s) = max_{a \in A(s)} \{r_{k}, \ p_{s}^a ⋅ f(t(s, a)) + (1 - p_{s}^a) ⋅ r_{k}^*\}$

## Implementação I

### Bibliotecas

In [1]:
import numpy as np

### Parâmetros

In [2]:
numero_perguntas = 15
N = numero_perguntas + 1

recompensas = [0, 150, 300, 450, 900, 1800, 2100, 2700, 3600, 4500, 9000, 18000, 36000, 72000, 144000, 300000]  # Tabela 1 do artigo, parâmetro r_k
recompensas_garantidas = [0, 0, 0, 0, 0, 1800, 1800, 1800, 1800, 1800, 9000, 9000, 9000, 9000, 9000, 300000] # Tabela 1 do artigo, parâmetro r_k^*

ajudas = ["50:50", "telefone", "plateia"]
qtd_ajudas = len(ajudas)

fatores_ajudas = {  # Tabela 2 do artigo, parâmetro c_k^i
    ajudas[0]: [0, 0.672, 0.698, 0.707, 0.711, 0.714, 0.716, 0.717, 0.718, 0.719, 0.719, 0.720, 0.720, 0.721, 0.721],
    ajudas[1]: [0, 0.527, 0.547, 0.554, 0.557, 0.559, 0.561, 0.562,  0.563, 0.563, 0.564, 0.564, 0.564, 0.565, 0.565],
    ajudas[2]: [0, 0.745, 0.773, 0.783, 0.788, 0.791, 0.793, 0.795,  0.796, 0.796, 0.797, 0.798, 0.798, 0.799, 0.799]
}

### Funções

In [3]:
# Variável p_k^*
def probabilidade_sem_ajuda(k):
  return max(0, 0.996 - 0.051 * (k - 1))

# Variável p_k^i
def ajustar_probabilidade(p, ajudas_usadas, k):
  if not ajudas_usadas:
    return p
  fator_ajuda_total = np.prod([fatores_ajudas[ajuda][k] for ajuda in ajudas_usadas])
  return 1 - (1 - p) * fator_ajuda_total

### Algoritmo de programação dinâmica

In [4]:
# Matriz para armazenar recompensas esperadas
recompensas_esperadas = np.zeros((N, 2**qtd_ajudas))

# Preenchendo o estágio de vitória (acertar todas as questões)
for idx in range(2**qtd_ajudas):
  recompensas_esperadas[N - 1, idx] = recompensas_garantidas[N - 1]

# Preenchendo as recompensas esperadas do penúltimo estágio até o primeiro estágio (abordagem top-down)
for estagio in range(N - 1, 0, -1):
  k = estagio - 1
  for estado in range(2**qtd_ajudas):  # Iterando sobre cada combinação de ajudas (000 até 111)
    p_sem_ajuda = probabilidade_sem_ajuda(estagio)

    recompensa_parar = recompensas[k]
    recompensa_responder = (p_sem_ajuda * recompensas_esperadas[estagio, estado] + (1 - p_sem_ajuda) * recompensas_garantidas[k])

    # Obtendo as ajudas disponíveis no estado atual
    ajudas_disponiveis = [ajuda for i, ajuda in enumerate(ajudas) if (estado >> i) & 1]

    if ajudas_disponiveis:
      p_com_ajuda = ajustar_probabilidade(p_sem_ajuda, ajudas_disponiveis, k)
      novo_estado = estado & ~(sum(1 << i for i, ajuda in enumerate(ajudas) if ajuda in ajudas_disponiveis))
      recompensa_responder = max(recompensa_responder, p_com_ajuda * recompensas_esperadas[estagio, novo_estado] + (1 - p_com_ajuda) * recompensas_garantidas[k])

    # Atualiza-se a recompensa esperada no estado atual
    recompensas_esperadas[k, estado] = max(recompensa_parar, recompensa_responder)

### Exibindo resultados na Questão 15

Ao comparar os resultados obtidos com a Tabela 3 do artigo, percebe-se que os resultados são satisfatórios e condizem com o esperado; embora haja uma pequena margem de erro.

In [5]:
print("\n--- Recompensas Esperadas na Questão 15 ---\n")
for estado in range(2**qtd_ajudas):
  ajudas_disponiveis = [ajudas[i] for i in range(qtd_ajudas) if (estado >> i) & 1]
  if ajudas_disponiveis:
    print(f"Ajuda(s): {' | '.join(ajudas_disponiveis)}")
  else:
    print(f"Sem ajudas disponíveis")
  print(f"  Recompensa esperada: {recompensas_esperadas[14, estado]:.2f}")
  print("----------------------------")


--- Recompensas Esperadas na Questão 15 ---

Sem ajudas disponíveis
  Recompensa esperada: 144000.00
----------------------------
Ajuda(s): 50:50
  Recompensa esperada: 149355.70
----------------------------
Ajuda(s): telefone
  Recompensa esperada: 181950.03
----------------------------
Ajuda(s): 50:50 | telefone
  Recompensa esperada: 214885.97
----------------------------
Ajuda(s): plateia
  Recompensa esperada: 144000.00
----------------------------
Ajuda(s): 50:50 | plateia
  Recompensa esperada: 179635.21
----------------------------
Ajuda(s): telefone | plateia
  Recompensa esperada: 205678.07
----------------------------
Ajuda(s): 50:50 | telefone | plateia
  Recompensa esperada: 231993.89
----------------------------


## Chegando à uma questão

O objetivo agora é encontrar uma estratégia ótima para maximizar a probabilidade de alcançar e responder corretamente a uma determinada questão.

### Novas Variáveis

* $w$: número da questão que queremos chegar e acertar;
* $g_i$: variável binária, onde $1$ se a ajuda $i$ é usada na questão $k$, 0 caso contrário;
* $p_{k, g_1, g_2, g_3}$: probabilidade de responder corretamente questão $k$, usando a ajuda $i$ se $g_i = 1$.

Percebe-se que $g_i$ é semelhante às variáveis $l_{i}$ e $l_{i}^´$, mas as diferenças são:
* $l_{i}$: indica se a ajuda está disponível no estado atual;
* $l_{i}^´$: serve para verificar/tirar a disponibilidade de $l{i}$;
* $g_i$: indica se a ajuda será usada neste momento específico.

Se em $p_{k, g_1, g_2, g_3}$ não é usada nenhuma ajuda, isso é o mesmo que $p_k^*$.

### Função Objetivo

$f(s)$ consiste em maximizar a probabilidade de chegar e acertar a questão $w$ a partir de um estado $s$.

Podemos avaliar a nova função objetivo da seguinte maneira:
* A máxima probabilidade de atingir e responder corretamente a questão número $w$, começando no estado $s$, é dada pelo maior valor entre as probabilidades das ações possíveis $a \in A(s)$, considerando a melhor combinação das ajudas $g_1, g_2, g_3$;
* Essa probabilidade é o produto da chance de responder corretamente a questão atual e da maior probabilidade de alcançar o objetivo a partir do estado de transição $t(a, s)$, que é o estado resultante de tomar a ação $a$ a partir do estado $s$;
* Deve-se atentar que cada estado de transição é necessário refletir quais ajudas restam para as próximas questões.

Matematicamente, temos:

$f(k, l_1, l_2, l_3) = max_{0 \ \le \ g_i \ \le \ l_i \\ g_i \ \in \ \mathbb{Z} \ ∀ i} \{p_{k, g_1, g_2, g_3} ⋅ f(k+1, l_1 - g_1, l_2 - g_2, l_3 - g_3)\}$

Se estivermos na questão $w + 1$, então $w$ foi respondida com sucesso, logo a probabilidade nesse caso é igual a $1$. Por isso, tem-se que:

$f(w + 1, l_1, l_2, l_3) = 1 \quad ∀ l_i \in \{0, 1\},\quad i = \{1, 2, 3\}$

## Implementação II

In [6]:
def alcancar_acertar_questao(w):
  if w == 0:
    return None

  # Matriz para armazenar as probabilidades máximas de alcançar a questão desejada
  probabilidades_acerto = np.zeros((w + 2, 2**qtd_ajudas))

  # Guardará a estrátegia que pode ser utilizada
  estrategia = np.full(w + 1, "", dtype=object)

  # Segundo o artigo, a primeira questão sempre é acertada
  for idx in range(2**qtd_ajudas):
    estrategia[0] = "nenhuma"
    probabilidades_acerto[0][idx] = 1.0

  # Caso base: Se for a primeira questão
  if w == 1:
    estrategia[1] = " | ".join(ajudas)
    return np.ones((w + 1, 2**qtd_ajudas)), estrategia

  # Se estivermos na questão w + 1, a probabilidade de sucesso é 1
  for idx in range(2**qtd_ajudas):
    probabilidades_acerto[w + 1, idx] = 1.0

  ajudas_disponiveis = ajudas.copy()

  # Algoritmo de programação dinâmica
  for k in range(w, 0, -1):
    melhor_probabilidade = 0.0
    melhor_ajuda = "nenhuma"

    for estado in range(2**qtd_ajudas):
      ajudas_atuais = [ajuda for i, ajuda in enumerate(ajudas) if (estado >> i) & 1]

      if not all(ajuda in ajudas_disponiveis for ajuda in ajudas_atuais):
        continue

      probabilidade_atual = probabilidade_sem_ajuda(k)
      if ajudas_atuais:
        probabilidade_atual = ajustar_probabilidade(probabilidade_atual, ajudas_atuais, k - 1)
      probabilidades_acerto[k, estado] = probabilidade_atual

      complemento_estado = ~estado & ((1 << len(ajudas)) - 1)
      ajudas_complementares = [ajuda for i, ajuda in enumerate(ajudas) if (complemento_estado >> i) & 1]

      probabilidade_acima = probabilidades_acerto[k + 1][complemento_estado]

      produto_probabilidade = probabilidade_atual * probabilidade_acima

      if produto_probabilidade > melhor_probabilidade:
        melhor_probabilidade = produto_probabilidade
        melhor_ajuda = " | ".join(ajudas_atuais) if ajudas_atuais else "nenhuma"

    estrategia[k] = melhor_ajuda

    for ajuda in melhor_ajuda.split(" | "):
      if ajuda in ajudas_disponiveis:
        ajudas_disponiveis.remove(ajuda)

    for idx in range(2**qtd_ajudas):
      probabilidades_acerto[k, idx] = melhor_probabilidade

  return probabilidades_acerto, estrategia

### Exibindo resultados

Ao comparar os resultados obtidos com a Tabela 5 e 6 do artigo, percebe-se que os resultados são bons e condizem com o esperado; embora haja uma razoável margem de erro nas questões intermediárias.

Essa margem de erro é justificada pela seguinte razão:
* O artigo estipula uma ordem de usar as ajudas: plateia, 50:50, telefone;
* Essa ordem não é seguida, pois na implementação é armazenada sempre a melhor probabilidade do estado atual, como é utilizado uma abordagem *top-down*, então a máxima probabilidade na questão $w$ sempre é utilizar as três ajudas simultaneamente, logo os demais estados $(\lt w)$ sempre não utilizam ajudas.

Codifiquei dessa forma, pois reaprovetei a ideia que fiz na implementação I, assim o código exige menos esforço computacional (pois pesquisa menos soluções), mas ao custo de não entregar uma probabilidade boa nas questões intermediárias.

O interessante seria tentar implementar uma abordagem *bottom-up* e comparar os resultados, porém isso não foi realizado.

In [7]:
print('\n### "Melhor" estratégia para alcançar e acertar qualquer questão ###\n')
for w in range(N - 1, 0, -1):
  print(f"\n--- Questão {w} ---\n")
  probabilidades_acerto, estrategia = alcancar_acertar_questao(w)

  for qi in range(0, w):
    print(f"QI: {qi + 1}")
    print(f"  Ajudas usadas: {estrategia[qi + 1]}")
  print(f"Probabilidade: {probabilidades_acerto[1, 3]:.3f}")
  print("----------------------------")


### "Melhor" estratégia para alcançar e acertar qualquer questão ###


--- Questão 15 ---

QI: 1
  Ajudas usadas: nenhuma
QI: 2
  Ajudas usadas: nenhuma
QI: 3
  Ajudas usadas: nenhuma
QI: 4
  Ajudas usadas: nenhuma
QI: 5
  Ajudas usadas: nenhuma
QI: 6
  Ajudas usadas: nenhuma
QI: 7
  Ajudas usadas: nenhuma
QI: 8
  Ajudas usadas: nenhuma
QI: 9
  Ajudas usadas: nenhuma
QI: 10
  Ajudas usadas: nenhuma
QI: 11
  Ajudas usadas: nenhuma
QI: 12
  Ajudas usadas: nenhuma
QI: 13
  Ajudas usadas: nenhuma
QI: 14
  Ajudas usadas: nenhuma
QI: 15
  Ajudas usadas: 50:50 | telefone | plateia
Probabilidade: 0.001
----------------------------

--- Questão 14 ---

QI: 1
  Ajudas usadas: nenhuma
QI: 2
  Ajudas usadas: nenhuma
QI: 3
  Ajudas usadas: nenhuma
QI: 4
  Ajudas usadas: nenhuma
QI: 5
  Ajudas usadas: nenhuma
QI: 6
  Ajudas usadas: nenhuma
QI: 7
  Ajudas usadas: nenhuma
QI: 8
  Ajudas usadas: nenhuma
QI: 9
  Ajudas usadas: nenhuma
QI: 10
  Ajudas usadas: nenhuma
QI: 11
  Ajudas usadas: nenhuma
QI: 